In [1]:
import geopandas as gpd
import os
import glob
import tools
import rasterio
from rasterio.enums import Resampling
import numpy as np
from rasterio.transform import from_bounds

### CATCHMENTS SHAPEFILE DETERMINES THE BOUNDS SUBSET (FROM KRYCKLAN DATABASE)

In [2]:
file = r'/Users/jpnousu/Library/CloudStorage/OneDrive-Valtion/Krycklan data/GIS/CATCHMENTS/Krycklan_catchments/Krycklan catchments.shp'
df = gpd.read_file(file)
subset = df.total_bounds
subset = subset.round(0)
# and 50m buffer
subset[0] = subset[0]-50
subset[1] = subset[1]-50
subset[2] = subset[2]+50
subset[3] = subset[3]+50

#### SUBSETTING SLU FOREST MAP AND FOREST BASIC DATA TO KRYCKLAN

In [4]:
# Forest basic
fd = r'/Users/jpnousu/Library/CloudStorage/OneDrive-Valtion/Krycklan data/GIS/FOREST_BASIC/'
out_fd = '/Users/jpnousu/Library/CloudStorage/OneDrive-Valtion/Krycklan data/GIS/FOREST_BASIC_KRYCKLAN/'
p = os.path.join(fd, '*.tif')
for file in glob.glob(p):
    out_fn = file.rpartition('/')[-1][:-4]
    out_fp = os.path.join(out_fd, out_fn) + '.tif'
    tools.open_raster_with_subset(file, out_fp, subset, plot=False, save_in='asc')

# SLU Forest map
fd = r'/Users/jpnousu/Library/CloudStorage/OneDrive-Valtion/Krycklan data/GIS/SLU_FOREST_MAP/2010/'
out_fd = '/Users/jpnousu/Library/CloudStorage/OneDrive-Valtion/Krycklan data/GIS/SLU_FOREST_MAP_KRYCKLAN/2010/'
p = os.path.join(fd, '*.tif')
for file in glob.glob(p):
    out_fn = file.rpartition('/')[-1][:-4]
    out_fp = os.path.join(out_fd, out_fn) + '.asc'
    tools.open_raster_with_subset(file, out_fp, subset, plot=False, save_in='asc')

#### Processing nan and canopy height to metric

In [5]:
fd = '/Users/jpnousu/Library/CloudStorage/OneDrive-Valtion/Krycklan data/GIS/FOREST_BASIC_KRYCKLAN/'
out_fd = '/Users/jpnousu/Library/CloudStorage/OneDrive-Valtion/Krycklan data/GIS/FOREST_BASIC_KRYCKLAN/processed/'
p = os.path.join(fd, '*.asc')
for file in glob.glob(p):
    out_fn = file.rpartition('/')[-1][:-4]
    out_fp = os.path.join(out_fd, out_fn) + '.asc'
    with rasterio.open(file) as src:
        data = src.read(1)
        meta = src.meta.copy()
        nodata_1 = meta['nodata']
        nodata_2 = -9999
        data[data == nodata_1] = nodata_2
        data[data == nodata_2] = 0.0
        meta.update({"nodata":nodata_2})
    with rasterio.open(out_fp, 'w', **meta) as dst:
        src = dst.write(data, 1)

fd = '/Users/jpnousu/Library/CloudStorage/OneDrive-Valtion/Krycklan data/GIS/SLU_FOREST_MAP_KRYCKLAN/2010/'
out_fd = '/Users/jpnousu/Library/CloudStorage/OneDrive-Valtion/Krycklan data/GIS/SLU_FOREST_MAP_KRYCKLAN/2010_processed/'
p = os.path.join(fd, '*.asc')
for file in glob.glob(p):
    out_fn = file.rpartition('/')[-1][:-4]
    out_fp = os.path.join(out_fd, out_fn) + '.asc'
    with rasterio.open(file) as src:
        data = src.read(1)
        meta = src.meta.copy()
        nodata_1 = meta['nodata']
        nodata_2 = -9999
        data[data == nodata_1] = nodata_2
        data[data == nodata_2] = 0.0
        meta.update({"nodata":nodata_2})
    with rasterio.open(out_fp, 'w', **meta) as dst:
        src = dst.write(data, 1)

file = r'/Users/jpnousu/Library/CloudStorage/OneDrive-Valtion/Krycklan data/GIS/FOREST_BASIC_KRYCKLAN/processed/sksMedelhöjd24.asc'
out_fp = r'/Users/jpnousu/Library/CloudStorage/OneDrive-Valtion/Krycklan data/GIS/FOREST_BASIC_KRYCKLAN/processed/canopy_height_10.asc'
with rasterio.open(file) as src:
    data = src.read(1)
    data = data.astype(np.float32)
    meta = src.meta.copy()
    meta.update(dtype='float32')  # Update metadata to match data type
    data = data / 10.
with rasterio.open(out_fp, 'w', **meta) as dst:
    src = dst.write(data, 1)

### CANOPY FRACTION AND LAI FRACTIONS FROM FOREST MAP

In [ ]:
vol_spruce_fp = r'/Users/jpnousu/Library/CloudStorage/OneDrive-Valtion/Krycklan data/GIS/SLU_FOREST_MAP_KRYCKLAN/2010_processed/SPRUCEVOL_XX_P_10.asc'
vol_pine_fp = r'/Users/jpnousu/Library/CloudStorage/OneDrive-Valtion/Krycklan data/GIS/SLU_FOREST_MAP_KRYCKLAN/2010_processed/PINEVOL_XX_P_10.asc'
vol_decid_fp = r'/Users/jpnousu/Library/CloudStorage/OneDrive-Valtion/Krycklan data/GIS/SLU_FOREST_MAP_KRYCKLAN/2010_processed/DECIDUOUSVOL_XX_P_10.asc'

lai_spruce_fp = r'/Users/jpnousu/Library/CloudStorage/OneDrive-Valtion/Krycklan data/GIS/SLU_FOREST_MAP_KRYCKLAN/2010_processed/LAI_FM_spruce.asc'
lai_pine_fp = r'/Users/jpnousu/Library/CloudStorage/OneDrive-Valtion/Krycklan data/GIS/SLU_FOREST_MAP_KRYCKLAN/2010_processed/LAI_FM_pine.asc'
lai_decid_fp = r'/Users/jpnousu/Library/CloudStorage/OneDrive-Valtion/Krycklan data/GIS/SLU_FOREST_MAP_KRYCKLAN/2010_processed/LAI_FM_decid.asc'

files = [vol_spruce_fp, vol_pine_fp, vol_decid_fp]
out_files = [lai_spruce_fp, lai_pine_fp, lai_decid_fp]
trees = ['spruce', 'pine', 'decid']

for idx, file in enumerate(files):
    with rasterio.open(file) as src:
        vol = src.read(1)
        vol = vol.astype(float)
        out_meta = src.meta.copy()
        nodata = out_meta['nodata']
        vol[vol == nodata] = 0.0
        LAI = stem_volume_to_LAI(vol, tree=trees[idx])
        out_meta.update({"nodata": -9999,
                        "dtype": np.float32,
                        })
        with rasterio.open(out_files[idx], 'w', **out_meta) as dst:
            out = dst.write(LAI, 1)

In [ ]:
lai_spruce_fp = r'/Users/jpnousu/Library/CloudStorage/OneDrive-Valtion/Krycklan data/GIS/SLU_FOREST_MAP_KRYCKLAN/2010_processed/LAI_FM_spruce.asc'
lai_pine_fp = r'/Users/jpnousu/Library/CloudStorage/OneDrive-Valtion/Krycklan data/GIS/SLU_FOREST_MAP_KRYCKLAN/2010_processed/LAI_FM_pine.asc'
lai_decid_fp = r'/Users/jpnousu/Library/CloudStorage/OneDrive-Valtion/Krycklan data/GIS/SLU_FOREST_MAP_KRYCKLAN/2010_processed/LAI_FM_decid.asc'
lai_conif_fp = r'/Users/jpnousu/Library/CloudStorage/OneDrive-Valtion/Krycklan data/GIS/SLU_FOREST_MAP_KRYCKLAN/2010_processed/LAI_FM_conif.asc'

laif_spruce_fp = r'/Users/jpnousu/Library/CloudStorage/OneDrive-Valtion/Krycklan data/GIS/SLU_FOREST_MAP_KRYCKLAN/2010_processed/LAIF_FM_spruce.asc'
laif_pine_fp = r'/Users/jpnousu/Library/CloudStorage/OneDrive-Valtion/Krycklan data/GIS/SLU_FOREST_MAP_KRYCKLAN/2010_processed/LAIF_FM_pine.asc'
laif_decid_fp = r'/Users/jpnousu/Library/CloudStorage/OneDrive-Valtion/Krycklan data/GIS/SLU_FOREST_MAP_KRYCKLAN/2010_processed/LAIF_FM_decid.asc'

with rasterio.open(lai_spruce_fp) as src1:
    lai_spruce = src1.read(1)
    out_meta = src1.meta.copy()
    nodata = out_meta['nodata']
with rasterio.open(lai_pine_fp) as src2:
    lai_pine = src2.read(1)
with rasterio.open(lai_decid_fp) as src3:
    lai_decid = src3.read(1)
    lai_tot = lai_spruce + lai_pine + lai_decid
    laif_decid = lai_decid / lai_tot
    laif_spruce = lai_spruce / lai_tot
    laif_pine = lai_pine / lai_tot
    laif_spruce[~np.isfinite(laif_spruce)] = 0
    laif_pine[~np.isfinite(laif_pine)] = 0
    laif_decid[~np.isfinite(laif_decid)] = 0
    
    with rasterio.open(laif_spruce_fp, 'w', **out_meta) as dst1:
        out1 = dst1.write(laif_spruce, 1)
    with rasterio.open(laif_pine_fp, 'w', **out_meta) as dst2:
        out2 = dst2.write(laif_pine, 1)
    with rasterio.open(laif_decid_fp, 'w', **out_meta) as dst3:
        out3 = dst3.write(laif_decid, 1)

### LAI (FROM HENRIK)

In [ ]:
fp = r'/Users/jpnousu/Library/CloudStorage/OneDrive-Valtion/Krycklan data/GIS/KRYCKLAN_LAI/LAI_Krycklan_ALS_2019.tif'
out_fp = r'/Users/jpnousu/Library/CloudStorage/OneDrive-Valtion/Krycklan data/GIS/KRYCKLAN_LAI/processed/LAI_Krycklan_ALS_2019_match_forestmap.asc'
match = r'/Users/jpnousu/Library/CloudStorage/OneDrive-Valtion/Krycklan data/GIS/SLU_FOREST_MAP_KRYCKLAN/2010/HEIGHT_XX_P_10.asc'
tools.reproj_match(infile=fp, match=match, outfile=out_fp, resampling_method='bilinear', save_in='asc')

In [ ]:
file = r'/Users/jpnousu/Library/CloudStorage/OneDrive-Valtion/Krycklan data/GIS/KRYCKLAN_LAI/processed/LAI_Krycklan_ALS_2019_match_forestmap.asc'
out_fp = r'/Users/jpnousu/Library/CloudStorage/OneDrive-Valtion/Krycklan data/GIS/KRYCKLAN_LAI/processed/LAI_Krycklan_ALS_2019_match_forestmap_nona.asc'

with rasterio.open(file) as src:
    data = src.read(1)
    meta = src.meta.copy()
    nodata_1 = meta['nodata']
    nodata_2 = -9999
    data[data == nodata_1] = nodata_2
    data[data == nodata_2] = 0.0
    data[data < 0] = 0.0
    meta.update({"nodata": nodata_2})
with rasterio.open(out_fp, 'w', **meta) as dst:
    src = dst.write(data, 1)

### LAI FRACTIONS * LAI TOT = LAI SPECIES

In [ ]:
fp = r'/Users/jpnousu/Library/CloudStorage/OneDrive-Valtion/Krycklan data/GIS/KRYCKLAN_LAI/processed/LAI_Krycklan_ALS_2019_match_forestmap_nona.asc'
lai_spruce_fp = r'/Users/jpnousu/Library/CloudStorage/OneDrive-Valtion/Krycklan data/GIS/KRYCKLAN_LAI/processed/LAI_spruce.asc'
lai_pine_fp = r'/Users/jpnousu/Library/CloudStorage/OneDrive-Valtion/Krycklan data/GIS/KRYCKLAN_LAI/processed/LAI_pine.asc'
lai_decid_fp = r'/Users/jpnousu/Library/CloudStorage/OneDrive-Valtion/Krycklan data/GIS/KRYCKLAN_LAI/processed/LAI_decid.asc'

with rasterio.open(fp) as src:
    lai_tot = src.read(1)
    nodata = src.meta.copy()['nodata']
    out_meta = src.meta.copy()
    lai_decid = lai_tot * laif_decid
    lai_spruce = lai_tot * laif_spruce
    lai_pine = lai_tot * laif_pine
    lai_decid[lai_tot == nodata] = nodata
    lai_spruce[lai_tot == nodata] = nodata
    lai_pine[lai_tot == nodata] = nodata
    
    lai_decid[(lai_decid < 0) & (lai_decid != nodata)] = 0
    lai_spruce[(lai_spruce < 0) & (lai_spruce != nodata)] = 0
    lai_pine[(lai_pine < 0) & (lai_pine != nodata)] = 0
    
    with rasterio.open(lai_spruce_fp, 'w', **out_meta) as dst:
        out = dst.write(lai_spruce, 1)
    with rasterio.open(lai_pine_fp, 'w', **out_meta) as dst:
        out = dst.write(lai_pine, 1)
    with rasterio.open(lai_decid_fp, 'w', **out_meta) as dst:
        out = dst.write(lai_decid, 1)

In [ ]:
vol_spruce_fp = r'/Users/jpnousu/Library/CloudStorage/OneDrive-Valtion/Krycklan data/GIS/SLU_FOREST_MAP_KRYCKLAN/2010/SPRUCEVOL_XX_P_10.asc'
vol_pine_fp = r'/Users/jpnousu/Library/CloudStorage/OneDrive-Valtion/Krycklan data/GIS/SLU_FOREST_MAP_KRYCKLAN/2010/PINEVOL_XX_P_10.asc'
vol_decid_fp = r'/Users/jpnousu/Library/CloudStorage/OneDrive-Valtion/Krycklan data/GIS/SLU_FOREST_MAP_KRYCKLAN/2010/DECIDUOUSVOL_XX_P_10.asc'

cf_out_fp = r'/Users/jpnousu/Library/CloudStorage/OneDrive-Valtion/Krycklan data/GIS/SLU_FOREST_MAP_KRYCKLAN/2010_processed/canopy_fraction.asc'

with rasterio.open(vol_spruce_fp) as src:
    vol = src.read(1)
    vol = vol.astype(float)
    out_meta = src.meta.copy()
    nodata = out_meta['nodata']
    vol[vol == nodata] = 0.0
    BA_spruce = tools.ba_from_vol(vol=vol, a=0.09)
with rasterio.open(vol_pine_fp) as src:
    vol = src.read(1)
    vol = vol.astype(float)
    out_meta = src.meta.copy()
    nodata = out_meta['nodata']
    vol[vol == nodata] = 0.0
    BA_pine = tools.ba_from_vol(vol=vol, a=0.12)
with rasterio.open(vol_decid_fp) as src:
    vol = src.read(1)
    vol = vol.astype(float)
    out_meta = src.meta.copy()
    nodata = out_meta['nodata']
    vol[vol == nodata] = 0.0
    BA_decid = tools.ba_from_vol(vol=vol, a=0.12)

    BA_tot = BA_spruce + BA_pine + BA_decid
    
    cf = tools.cf_from_ba(ba=BA_tot)
    
    out_meta.update({"nodata": -9999,
                    "dtype": np.float32,
                    })
with rasterio.open(cf_out_fp, 'w', **out_meta) as dst:
    out = dst.write(cf, 1)

#### REPROJECT 

In [2]:
ref = r'/Users/jpnousu/Library/CloudStorage/OneDrive-Valtion/Krycklan data/GIS/FOREST_BASIC_KRYCKLAN/processed/canopy_height_10.asc'

cf = r'/Users/jpnousu/Library/CloudStorage/OneDrive-Valtion/Krycklan data/GIS/SLU_FOREST_MAP_KRYCKLAN/2010_processed/canopy_fraction.asc'
lai_spruce_fp = r'/Users/jpnousu/Library/CloudStorage/OneDrive-Valtion/Krycklan data/GIS/KRYCKLAN_LAI/processed/LAI_spruce.asc'
lai_pine_fp = r'/Users/jpnousu/Library/CloudStorage/OneDrive-Valtion/Krycklan data/GIS/KRYCKLAN_LAI/processed/LAI_pine.asc'
lai_decid_fp = r'/Users/jpnousu/Library/CloudStorage/OneDrive-Valtion/Krycklan data/GIS/KRYCKLAN_LAI/processed/LAI_decid.asc'

files = [cf, lai_spruce_fp, lai_pine_fp, lai_decid_fp]
resolutions = [10, 20, 40, 80, 160]

# Reproject all rasters to match 10m ref
for file in files:
    dir_name = os.path.dirname(file)
    base_name = os.path.basename(file)
    name, ext = os.path.splitext(base_name)

    file_10 = os.path.join(dir_name, f"{name}_10{ext}")
    tools.reproj_match(infile=file, match=ref, outfile=file_10, resampling_method='nearest', save_in='asc')  # or 'bilinear' / 'average'

# Add the reference raster to be resampled
all_to_resample = [ref] + [
    os.path.join(os.path.dirname(f), f"{os.path.splitext(os.path.basename(f))[0]}_10{os.path.splitext(f)[1]}")
    for f in files
]

# Resample everything to coarser resolutions
for file_10 in all_to_resample:
    dir_name = os.path.dirname(file_10)
    base_name = os.path.basename(file_10)
    name, ext = os.path.splitext(base_name)

    for res in resolutions[1:]:  # Skip 10m
        scale = res // 10

        with rasterio.open(file_10) as src:
            scale = res // 10
            res_val = scale * 10
        
            # Get original bounds and dimensions
            left, bottom, right, top = src.bounds
        
            # Calculate new dimensions based on desired resolution
            new_width = int((right - left) / res_val)
            new_height = int((top - bottom) / res_val)
        
            # Force-align the bounds exactly with bottom-left fixed
            # This ensures same xllcorner and yllcorner
            new_right = left + new_width * res_val
            new_top = bottom + new_height * res_val
        
            # Compute transform from bounds
            new_transform = from_bounds(left, new_top, new_right, bottom, new_width, new_height)
        
            # Resample
            data_resampled = src.read(
                out_shape=(src.count, new_height, new_width),
                resampling=Resampling.average
            )
        
            out_meta = src.meta.copy()
            out_meta.update({
                'height': new_height,
                'width': new_width,
                'transform': new_transform
            })
        
            out_path = os.path.join(dir_name, f"{name.replace('_10', f'_{res}')}{ext}")
            with rasterio.open(out_path, 'w', **out_meta) as dst:
                dst.write(data_resampled)

        print(f"Created: {out_path}")

Coregistered to shape: 1151 1031 
 Affine | 10.00, 0.00, 726726.00|
| 0.00,-10.00, 7138700.00|
| 0.00, 0.00, 1.00|
Coregistered to shape: 1151 1031 
 Affine | 10.00, 0.00, 726726.00|
| 0.00,-10.00, 7138700.00|
| 0.00, 0.00, 1.00|
Coregistered to shape: 1151 1031 
 Affine | 10.00, 0.00, 726726.00|
| 0.00,-10.00, 7138700.00|
| 0.00, 0.00, 1.00|
Coregistered to shape: 1151 1031 
 Affine | 10.00, 0.00, 726726.00|
| 0.00,-10.00, 7138700.00|
| 0.00, 0.00, 1.00|
Created: /Users/jpnousu/Library/CloudStorage/OneDrive-Valtion/Krycklan data/GIS/FOREST_BASIC_KRYCKLAN/processed/canopy_height_20.asc
Created: /Users/jpnousu/Library/CloudStorage/OneDrive-Valtion/Krycklan data/GIS/FOREST_BASIC_KRYCKLAN/processed/canopy_height_40.asc
Created: /Users/jpnousu/Library/CloudStorage/OneDrive-Valtion/Krycklan data/GIS/FOREST_BASIC_KRYCKLAN/processed/canopy_height_80.asc
Created: /Users/jpnousu/Library/CloudStorage/OneDrive-Valtion/Krycklan data/GIS/FOREST_BASIC_KRYCKLAN/processed/canopy_height_160.asc
Created